

### Load libs

In [1]:
import pandas as pd
import io
import requests
import json
import numpy as np
import time
import os
import re

In [2]:
n = 500
pd.set_option('display.max_columns', n)
pd.set_option('display.max_rows', n)
pd.set_option('display.max_colwidth', -1)

/anaconda3/envs/kn/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


### Load data

In [3]:
# detailed json
folder = 'data/json/securitizations/detailed/'
files = [f for f in os.listdir(folder) if '.json' in f and '._' not in f]
files.sort()
file = files[-1]
path = folder + file
with open(path) as f:
    files_dict = json.load(f)
keys = list(files_dict.keys())
use_keys = [k for k in keys if k != 'total_securitizations']


### Grab data

In [4]:
term = 'AmeriCredit Automobile Receivables Trust 2020-1 Data Tape'
securitization = [file for file in use_keys if term in file][0]
securitization

'AmeriCredit Automobile Receivables Trust 2020-1 Data Tape.csv'

In [5]:
sub_dict = files_dict[securitization]

In [6]:
s_keys = list(sub_dict.keys())
s_keys = [k for k in s_keys if k != 'releases']


In [7]:
use_len = len(s_keys)
use_len

12

### Combine data tapes

In [8]:
collected = []
broke = []
master_list = []
for file in s_keys:
    print('reading {}'.format(file))
    collected.append(file)
    col_len = len(collected)
    url = sub_dict[file]
    print(url)
    try:
        init = requests.get(url).content
        df = pd.read_csv(io.StringIO(init.decode('utf-8')))
        df['dataset_name'] = file.split('.csv')[0]
        master_list.append(df)
        print(round(col_len / use_len, 3))
    except:
        broke.append(file)
        print('ERROR!!!!! {}'.format(file))
    time.sleep(2)
    print('-------------------------------------')
    

reading 2021.01.22_AmeriCredit Automobile Receivables Trust 2020-1 Data Tape.csv
https://finsight.com/api/download-csv?fileId=74236
0.083
-------------------------------------
reading 2020.12.22_AmeriCredit Automobile Receivables Trust 2020-1 Data Tape.csv
https://finsight.com/api/download-csv?fileId=72267
0.167
-------------------------------------
reading 2020.11.23_AmeriCredit Automobile Receivables Trust 2020-1 Data Tape.csv
https://finsight.com/api/download-csv?fileId=70206
0.25
-------------------------------------
reading 2020.10.23_AmeriCredit Automobile Receivables Trust 2020-1 Data Tape.csv
https://finsight.com/api/download-csv?fileId=68375
0.333
-------------------------------------
reading 2020.09.23_AmeriCredit Automobile Receivables Trust 2020-1 Data Tape.csv
https://finsight.com/api/download-csv?fileId=65607
0.417
-------------------------------------
reading 2020.08.24_AmeriCredit Automobile Receivables Trust 2020-1 Data Tape.csv
https://finsight.com/api/download-csv?fi

/anaconda3/envs/kn/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (38) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0.667
-------------------------------------
reading 2020.05.22_AmeriCredit Automobile Receivables Trust 2020-1 Data Tape.csv
https://finsight.com/api/download-csv?fileId=56581
0.75
-------------------------------------
reading 2020.04.23_AmeriCredit Automobile Receivables Trust 2020-1 Data Tape.csv
https://finsight.com/api/download-csv?fileId=54622
0.833
-------------------------------------
reading 2020.04.23_AmeriCredit Automobile Receivables Trust 2020-1 Data Tape.csv_1
https://finsight.com/api/download-csv?fileId=54618
0.917
-------------------------------------
reading 2020.02.24_AmeriCredit Automobile Receivables Trust 2020-1 Data Tape.csv
https://finsight.com/api/download-csv?fileId=50876
1.0
-------------------------------------


### Quick processing

In [9]:
master_df = pd.DataFrame()
count = 0
for df in master_list:
    count = count + 1
    print(count)
    master_df = master_df.append(df)
    

1
2
3
4
5
6
7
8
9
10
11
12


In [10]:
master_df = master_df.reset_index(drop = True)


In [11]:
master_df['obligorCreditScore'].replace('None', np.nan, inplace = True)
master_df['obligorCreditScore'] = master_df['obligorCreditScore'].astype(float)
master_df['obligorCreditScore'].mean()

577.6733481649509

In [12]:
master_df.shape

(554258, 73)

### Export

In [13]:
e_folder = 'data/transaction/'
e_file = '{}.csv'.format(term)
e_path = e_folder + e_file
e_path

'data/transaction/AmeriCredit Automobile Receivables Trust 2020-1 Data Tape.csv'

In [14]:
master_df.to_csv(e_path, index = False)

In [15]:
print('complete...')

complete...


### End